### Create materialization config objects

In [ ]:
DROP TABLE [gold_sapecc_warehouse].[dbo].[accessrule_new]
GO



In [ ]:
-- Create log table
CREATE TABLE utilities.materialization_log (
    medallion_name varchar(100),
    schema_name varchar(100),
    table_name varchar(100),
    job_run_timestamp datetime2(6), 
    run_id INT,
    run_step varchar(100),
    run_timestamp datetime2(6), 
    record_count INT,
    step_fail varchar(max),
    statement_text varchar(max)
);

--Create load table
CREATE TABLE utilities.materialization_load (
    medallion_name varchar(100),
    input_source_schema varchar(100),
    input_source varchar(100),
    output_target_schema varchar(100), 
    output_target varchar(100),
    load_option varchar(100),
    load_type varchar(100),
    is_active bit,
    date_created datetime2(6),
    last_execution datetime2(6),
    last_successful_execution datetime2(6)
    );

-- create control table
CREATE TABLE utilities.materialization_control (
    medallion_name varchar(100),
    etl_database varchar(100),
    etl_view varchar(100),
    status_code varchar(100),
    extract_start_timestamp datetime2(6),
    extract_end_timestamp datetime2(6),
    script_header varchar(100),
    last_script_generate_timestamp datetime2(6),
    control_start_timestamp datetime2(6),
    load_option varchar(100),
    is_active bit
)



### Create input and target objects for demo

In [ ]:
--input object
alter view materialized_etl.acct_doc_hdr_etl as
select 
    client,
    company_code,
    acct_doc_nbr,
    cast(gjahr  as varchar(100)) fiscal_year,
    cast(blart  as varchar(100)) blart,
    cast(bldat  as datetime2(6)) bldat,
    cast(budat  as datetime2(6)) budat,
    cast(monat  as varchar(100)) monat,
    cast(waers  as varchar(100)) waers,
    cast(xblnr  as varchar(100)) xblnr,
    cast(bktxt  as varchar(100)) bktxt,
    cast(stjah  as varchar(100)) stjah,
    cast(stblg  as varchar(100)) stblg,
    cast(xnetb  as varchar(100)) xnetb,
    action_type,
    row_insert_timestamp,
    row_update_timestamp
from silver_sapecc_lakehouse.materialized_t.acct_doc_hdr
where company_code in ('2000','3000','1000')

--target object
create TABLE [materialized_t].[acct_doc_hdr]
(
    client varchar(3) not null,
    company_code varchar(4) not null,
    acct_doc_nbr varchar(10) not null,
    fiscal_year varchar(100),
    blart varchar(100),
    bldat  datetime2(6),
    budat  datetime2(6),
    monat  varchar(100),
    waers  varchar(100),
    xblnr  varchar(100),
    bktxt  varchar(100),
    stjah  varchar(100),
    stblg  varchar(100),
    xnetb  varchar(100),
    action_type char(1),
    row_insert_timestamp datetime2(6),
    row_update_timestamp datetime2(6)
)
GO

### materialization log view

In [ ]:
-- create the final view by union utilities and gold log table

create view utilities.vw_materialization_log as
select 
    [lakehouse_name] system_name,
    [schema_name],
    [table_name],
    [job_run_timestamp],
    [run_id],
    [run_step],
    [run_timestamp],
    [record_count],
    [step_fail],
    [statement_text]
from utilities_lakehouse.dbo.materialization_log --silver 
union all 
select
    [medallion_name] system_name,
    [schema_name],
    [table_name],
    [job_run_timestamp],
    [run_id],
    [run_step],
    [run_timestamp],
    [record_count],
    [step_fail],
    [statement_text]
from utilities.materialization_log --gold

In [ ]:
-- select * from the final log view
select * from utilities.vw_materialization_log ;

select top 100 * from utilities.vw_materialization_log
where table_name = 'acct_doc_hdr_etl'
order by job_run_timestamp desc, run_id desc


### Insert values into materialization load table

In [2]:
insert into utilities.materialization_load
(medallion_name,input_source_schema, input_source, output_target_schema, output_target, load_option,is_active, date_created )
values('gold_sapecc_warehouse', 'materialized_etl', 'porduct_etl', 'materialized_t', 'products', 'TR', 0, sysdatetime())

In [3]:
select * from utilities.materialization_load